# Parallelization Lab

In this lab, you will be leveraging several concepts you have learned to obtain a list of links from a web page and crawl and index the pages referenced by those links - both sequentially and in parallel. Follow the steps below to complete the lab.

## Step 1: Use the requests library to retrieve the content from the URL below.

In [1]:
import requests

url = 'https://en.wikipedia.org/wiki/Data_science'

In [14]:
r = requests.get(url)
r.status_code
#r.content

200

## Step 2: Use BeautifulSoup to extract a list of all the unique links on the page.

In [127]:
from bs4 import BeautifulSoup as bs

In [138]:
#Extraer las etiquetas <a> con href
sopa = bs(r.text,'lxml')
links=[a for a in sopa.find_all('a', href=True)]
print('Todas las ligas encontradas en las etiquetas a:' ,len(links))
links = set(links)
links = list(links)
print('Ligas despùes de eliminar duplicados', len(links))

Todas las ligas encontradas en las etiquetas a: 470
Ligas despùes de eliminar duplicados 414


## Step 3: Use list comprehensions with conditions to clean the link list.

There are two types of links, absolute and relative. Absolute links have the full URL and begin with http while relative links begin with a forward slash (/) and point to an internal page within the wikipedia.org domain. Clean the respective types of URLs as follows.

- Absolute Links: Create a list of these and remove any that contain a percentage sign (%).
- Relativel Links: Create a list of these, add the domain to the link so that you have the full URL, and remove any that contain a percentage sign (%).
- Combine the list of absolute and relative links and ensure there are no duplicates.

In [222]:
domain = 'http://wikipedia.org'

In [223]:
#Absolute Links  - Extraer las etiquetas "a" con class "external text"
sopa = bs(r.text,'lxml')
links_abs=[a for a in sopa.find_all('a', {'class':'external text'})]
print('Links extraidos de la sopa:',len(links_abs))


Links extraidos de la sopa: 69


In [224]:
#Quitar etiquetas y href y title con su texto
for i in range(len(links_abs)):
    links_abs[i] = str(links_abs[i])
    links_abs[i] = links_abs[i].replace('<a class="external text" href="','')
    fin = links_abs[i].find('"')
    links_abs[i] = links_abs[i][:fin]
print('Totalidad de Links absolutos:',len(links_abs))
#Quirar las que tinen %
for i in range(len(links_abs)):
    if links_abs[i].find('%') >= 0:
        links_abs[i] = False

for i in range(links_abs.count(False)):
    links_abs.remove(False)

print('Totalidad de Links absolutos sin %:',len(links_abs))

links_abs = set(links_abs)
links_abs = list(links_abs)
print('Links absolutos únicos:',len(links_abs))    


Totalidad de Links absolutos: 69
Totalidad de Links absolutos sin %: 60
Links absolutos únicos: 58


In [225]:
#Relative - Links: Marcar como false todo lo que no empieze con <a href=
sopa = bs(r.text,'lxml')
links=[a for a in sopa.find_all('a', href=True)]
print('Links extraidos de la sopa:',len(links))

Links extraidos de la sopa: 470


In [226]:
#Marcar los que no empiezan con '<a href="/wiki/' y que tienen %

for i in range(len(links)):
    links[i]=str(links[i])
    if links[i].find('<a href="/wiki/') !=0 or links[i].find('%') >= 0:
        links[i]=False

#Remover de la lista los False
for i in range(links.count(False)):
    links.remove(False)
print('Despues de quitar los que no son ligas relativas y que tienen % quedan:',len(links))

Despues de quitar los que no son ligas relativas y que tienen % quedan: 205


In [227]:
#Quitar etiquetas y href y title con su texto
for i in range(len(links)):
    links[i]=links[i].replace('<a href="',domain)
    fin=links[i].find('"')
    links[i]=links[i][:fin]

In [228]:
print('Links relativos extraidos:',len(links))
links = set(links)
links = list(links)
print('Links relativos unicos:',len(links))

Links relativos extraidos: 205
Links relativos unicos: 157


In [229]:
for i in range(len(links_abs)):
    links.append(links_abs[i]) 
print('Total de links:',len(links))


Total de links: 215


In [230]:
links

['http://wikipedia.org/wiki/Open_science',
 'http://wikipedia.org/wiki/Relevance_vector_machine',
 'http://wikipedia.org/wiki/Artificial_neural_network',
 'http://wikipedia.org/wiki/Temporal_difference_learning',
 'http://wikipedia.org/wiki/Machine_Learning_(journal)',
 'http://wikipedia.org/wiki/Distributed_computing',
 'http://wikipedia.org/wiki/Statistical_learning_theory',
 'http://wikipedia.org/wiki/Jeff_Hammerbacher',
 'http://wikipedia.org/wiki/Graphical_model',
 'http://wikipedia.org/wiki/Pattern_recognition',
 'http://wikipedia.org/wiki/New_York_University',
 'http://wikipedia.org/wiki/Information_explosion',
 'http://wikipedia.org/wiki/DBSCAN',
 'http://wikipedia.org/wiki/Predictive_modelling',
 'http://wikipedia.org/wiki/Social_science',
 'http://wikipedia.org/wiki/Computational_learning_theory',
 'http://wikipedia.org/wiki/Ensemble_learning',
 'http://wikipedia.org/wiki/Hierarchical_clustering',
 'http://wikipedia.org/wiki/David_Donoho',
 'http://wikipedia.org/wiki/Feature_

## Step 4: Use the os library to create a folder called *wikipedia* and make that the current working directory.

In [232]:
import os

In [ ]:
os.mkdir('./wikipedia')

In [241]:
os.getcwd()

'/Users/al/Documents/ironhack/datamex0120/module-1/lab-parallelization/your-code'

In [242]:
os.chdir('./wikipedia')

In [243]:
os.getcwd()

'/Users/al/Documents/ironhack/datamex0120/module-1/lab-parallelization/your-code/wikipedia'

## Step 5: Write a function called index_page that accepts a link and does the following.

- Tries to request the content of the page referenced by that link.
- Slugifies the filename using the `slugify` function from the [python-slugify](https://pypi.org/project/python-slugify/) library and adds a .html file extension.
    - If you don't already have the python-slugify library installed, you can pip install it as follows: `$ pip install python-slugify`.
    - To import the slugify function, you would do the following: `from slugify import slugify`.
    - You can then slugify a link as follows `slugify(link)`.
- Creates a file in the wikipedia folder using the slugified filename and writes the contents of the page to the file.
- If an exception occurs during the process above, just `pass`.

In [ ]:
from slugify import slugify

In [244]:
!pip3 install python-slugify

     |████████████████████████████████| 81kB 936kB/s eta 0:00:01
  Created wheel for python-slugify: filename=python_slugify-4.0.0-py2.py3-none-any.whl size=5487 sha256=dff0bc9748d6bec23b8b34a1cdcdb288026eb3f4e579ce0767108b8d3a09667d
  Stored in directory: /Users/al/Library/Caches/pip/wheels/11/94/81/312969455540cb0e6a773e5d68a73c14128bfdfd4a7969bb4f
Successfully built python-slugify


In [245]:
from slugify import slugify

In [249]:
print(links[0])
print(slugify(links[0])+'.html')

http://wikipedia.org/wiki/Open_science
http-wikipedia-org-wiki-open-science.html


In [258]:
f=open('name.txt','w+')
f.write('Esto es una linea')
f.close()

In [ ]:
from tqdm import tqdm
def linkstofile(links,print_results=True):
    contex=0
    contok=0
    contop=0
    contre=0
    contwr=0
    contcl=0
    #tqdm._instances.clear()
    for i in tqdm(links):
        try:
            #crear archivo con slufied name
            f = open(slugify(i)+'.html','w+')
            contop+=1
            #request
            r = requests.get(i)
            contre+=1
            #escribir resultado del request en el archivo
            f.write(r.text)
            contwr=+1
            f.close()
            contok+=1
        except:
            contex+=1
    
    if print_results == True:
        print('extraidas: {} \nabortadas: {}\nopen:{}\nrequest\nwrite{}:{}'
              .format(contok,contex,contop,contre,contwr))

## Step 6: Sequentially loop through the list of links, running the index_page function each time.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [ ]:
%%time 
linkstofile(links)

 57%|█████▋    | 123/215 [01:46<01:19,  1.16it/s]

## Step 7: Perform the page indexing in parallel and note the difference in performance.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [ ]:
import multiprocessing as mp

In [ ]:
pool = mp.Pool()

In [ ]:
%%time
pool(linkstofile(links))